In [1]:
# !pip install terratorch -q

In [6]:
import sys
sys.path.append('/kaggle/input/terratorch3')

In [7]:
import albumentations
from albumentations.pytorch import ToTensorV2
import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from terratorch.datamodules import GenericNonGeoSegmentationDataModule
from terratorch.tasks import SemanticSegmentationTask
import zipfile
import glob
import os
import numpy as np
import rasterio
import pandas as pd

/kaggle/input/terratorch3/pretrainedmodels/models/dpn.py:255: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if block_type is 'proj':
/kaggle/input/terratorch3/pretrainedmodels/models/dpn.py:258: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif block_type is 'down':
/kaggle/input/terratorch3/pretrainedmodels/models/dpn.py:262: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert block_type is 'normal'
2025-09-17 14:16:54.510426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758118614.693982      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758118614.759397      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registere

In [8]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [9]:
# rm -rf /kaggle/working/crop_17_bands

In [11]:
import numpy as np
import rasterio
import os
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

def calculate_spectral_indices(src):
    """
    Calculates 22 spectral indices from a rasterio source object.
        
    Assumes standard Sentinel-2 band ordering:
    B2=Blue, B3=Green, B4=Red, B5=RE1, B6=RE2, B8=NIR, B11=SWIR1, B12=SWIR2
    """
    # Read required bands into variables with a small epsilon to avoid division by zero
    epsilon = 1e-8
    bands = src.read().astype(np.float32)
    
    # Sentinel-2 Band Mapping (1-based index to 0-based array index)
    blue = bands[1]   # b02
    green = bands[2]  # b03
    red = bands[3]    # b04
    re1 = bands[4]    # b05 (Red-Edge 1)
    re2 = bands[5]    # b06 (Red-Edge 2)
    b7 = bands[6]
    nir = bands[7]    # b08 (NIR)
    swir1 = bands[10] # b11 (SWIR 1)
    swir2 = bands[11] # b12 (SWIR 2)

    # --- Calculate Indices ---
    # Suppress runtime warnings for invalid divisions, which we handle with np.nan_to_num
    with np.errstate(divide='ignore', invalid='ignore'):
        # --- Original 11 Indices ---
        ndvi = (nir - red) / (nir + red + epsilon)
        evi = 2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1 + epsilon)
        L = 0.5
        savi = ((nir - red) / (nir + red + L + epsilon)) * (1 + L)
        ndmi = (nir - swir1) / (nir + swir1 + epsilon)
        gvmi = ((nir + 0.1) - (swir2 + 0.02)) / ((nir + 0.1) + (swir2 + 0.02) + epsilon)
        ndre1 = (nir - re1) / (nir + re1 + epsilon)
        ndre2 = (nir - re2) / (nir + re2 + epsilon)
        ndyi = (green - blue) / (green + blue + epsilon)
        psri = (red - blue) / (re2 + epsilon)
        gndvi = (nir - green) / (nir + green + epsilon)
        rep = 705 + 35 * (((red + b7) / 2) - re1) / (re2 - re1 + epsilon)

        # --- START: Added 11 New Indices ---
        osavi = (nir - red) / (nir + red + 0.16 + epsilon)
        wdrvi = (0.1 * nir - red) / (0.1 * nir + red + epsilon)
        mcari = (re1 - red - 0.2 * (re1 - green)) * (re1 / (red + epsilon))
        tcari = 3 * ((re1 - red) - 0.2 * (re1 - green) * (re1 / (red + epsilon)))
        
        # CCCI is a ratio of NDRE1 and NDVI
        ccci_numerator = (nir - re1) / (nir + re1 + epsilon) # NDRE1
        ccci_denominator = (nir - red) / (nir + red + epsilon) # NDVI
        ccci = ccci_numerator / (ccci_denominator + epsilon)

        mtci = (re2 - re1) / (re1 - red + epsilon)
        cire = (nir / (re1 + epsilon)) - 1
        sipi = (nir - blue) / (nir - red + epsilon)
        bsi = ((swir1 + red) - (nir + blue)) / ((swir1 + red) + (nir + blue) + epsilon)
        exg = 2 * green - red - blue
        rdvi = (nir - red) / np.sqrt(nir + red + epsilon)
        # Note: NDVIre is the same formula as NDRE1, so it is not added again.
        # --- END: Added 11 New Indices ---

    indices = [
        # Original
        ndvi, evi, savi, ndmi, gvmi, ndre1, ndre2,
        ndyi, psri, gndvi, rep,
        # New
        osavi, wdrvi, mcari, tcari, ccci, mtci,
        cire, sipi, bsi, exg, rdvi
    ]
    
    # Handle potential NaN/Inf values that may have resulted from division issues
    indices_cleaned = [np.nan_to_num(idx) for idx in indices]
    
    return np.array(indices_cleaned, dtype=np.float32)

def select_add_indices_and_save(input_path, output_path, band_indices_to_keep):
    """
    Selects 6 bands, calculates 22 indices from the original 12 bands,
    and saves a new 28-band TIFF.
    """
    try:
        with rasterio.open(input_path) as src:
            if src.count < 12:
                print(f"Error: Input file {input_path} has {src.count} bands. At least 12 are required.")
                return False

            # Read the 6 bands you want to keep
            selected_bands = src.read(band_indices_to_keep)
            
            # Calculate the 22 new index bands from the full source
            index_bands = calculate_spectral_indices(src)

            # Stack the 6 selected bands and the 22 new index bands
            all_28_bands = np.vstack((selected_bands.astype(np.float32), index_bands))

            # Update profile for the new file
            profile = src.profile.copy()
            profile.update(
                count=34, # UPDATED BAND COUNT
                dtype=np.float32 # Use float32 to accommodate index values
            )

            # Create the output directory if it doesn't exist
            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            # Write the new 28-band TIFF file
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(all_28_bands)
        
        return True

    except Exception as e:
        print(f"An error occurred while processing {input_path}: {str(e)}")
        return False

def process_directory(input_dir, output_dir, band_indices_to_keep, pattern="*.tif"):
    """
    Processes all TIFFs in a directory to create 28-band files.
    """
    input_path = Path(input_dir)
    output_path = Path(output_dir)

    if not input_path.exists():
        print(f"Error: Input directory '{input_dir}' does not exist!")
        return

    output_path.mkdir(parents=True, exist_ok=True)
    tiff_files = list(input_path.glob(pattern))
    
    if not tiff_files:
        print(f"No TIFF files found matching pattern '{pattern}' in {input_dir}")
        return

    print(f"Found {len(tiff_files)} TIFF files to process...")
    successful, failed = 0, 0

    for tiff_file in tiff_files:
        output_file = output_path / tiff_file.name
        if select_add_indices_and_save(str(tiff_file), str(output_file), band_indices_to_keep):
            successful += 1
        else:
            failed += 1

    print("\nProcessing complete!")
    print(f"Successfully processed: {successful} files")
    print(f"Failed: {failed} files")

# --- Example usage and configuration ---
if __name__ == "__main__":
    # The bands selected to be kept in the final file.
    # The calculation step still uses all original bands as needed.
    BANDS_TO_SELECT = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] # Blue, Green, Red, NIR, SWIR1, SWIR2

    base_dir = '/kaggle/input/crop-dataset/track1-india-al-impact-gen-ai-hackathon'
    out_dir = '/kaggle/working/crop_34_bands' # UPDATED output directory name
    sub_dirs = ['train', 'val', 'test']
    
    for sub in sub_dirs:
        input_directory = os.path.join(base_dir, sub, 'inputs')
        output_directory = os.path.join(out_dir, sub, 'inputs')

        print("-" * 50)
        print(f"Starting batch processing for: {sub}")
        print(f"Input directory: {input_directory}")
        print(f"Output directory: {output_directory}")
        print(f"Keeping original band indices: {BANDS_TO_SELECT}")
        
        process_directory(input_directory, output_directory, BANDS_TO_SELECT)

    print("\nAll script operations completed!")


--------------------------------------------------
Starting batch processing for: train
Input directory: /kaggle/input/crop-dataset/track1-india-al-impact-gen-ai-hackathon/train/inputs
Output directory: /kaggle/working/crop_34_bands/train/inputs
Keeping original band indices: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Found 928 TIFF files to process...

Processing complete!
Successfully processed: 928 files
Failed: 0 files
--------------------------------------------------
Starting batch processing for: val
Input directory: /kaggle/input/crop-dataset/track1-india-al-impact-gen-ai-hackathon/val/inputs
Output directory: /kaggle/working/crop_34_bands/val/inputs
Keeping original band indices: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Found 119 TIFF files to process...

Processing complete!
Successfully processed: 119 files
Failed: 0 files
--------------------------------------------------
Starting batch processing for: test
Input directory: /kaggle/input/crop-dataset/track1-india-al-impact-gen-

In [12]:
import shutil
import os

def copy_folder(source_folder, destination_folder):
    """
    Copies an entire folder from a source path to a destination path.

    Args:
        source_folder (str): The path to the source folder.
        destination_folder (str): The path to the destination folder.
    """
    
    try:
        # Check if the destination folder already exists
        if os.path.exists(destination_folder):
            print(f"Warning: Destination folder '{destination_folder}' already exists. "
                  "If you want to overwrite, you might need to remove it first.")
            # Optionally, you could add logic here to remove the existing destination
            # shutil.rmtree(destination_folder)
            # print(f"Removed existing destination folder: {destination_folder}")
        
        # Copy the entire directory tree
        shutil.copytree(source_folder, destination_folder)
        print(f"Folder '{source_folder}' copied successfully to '{destination_folder}'.")
    except FileExistsError:
        print(f"Error: Destination folder '{destination_folder}' already exists. "
              "Use `dirs_exist_ok=True` in shutil.copytree to allow overwriting (Python 3.8+), "
              "or remove the destination folder first.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
base_dir = '/kaggle/input/crop-dataset/track1-india-al-impact-gen-ai-hackathon'
out_dir = '/kaggle/working/crop_34_bands' # Changed output dir name for clarity
sub_dirs = ['train', 'val', 'test']

for sub in sub_dirs:
    input_directory = os.path.join(base_dir, sub, 'labels')
    output_directory = os.path.join(out_dir, sub, 'labels')

    copy_folder(input_directory, output_directory)

Folder '/kaggle/input/crop-dataset/track1-india-al-impact-gen-ai-hackathon/train/labels' copied successfully to '/kaggle/working/crop_34_bands/train/labels'.
Folder '/kaggle/input/crop-dataset/track1-india-al-impact-gen-ai-hackathon/val/labels' copied successfully to '/kaggle/working/crop_34_bands/val/labels'.
Folder '/kaggle/input/crop-dataset/track1-india-al-impact-gen-ai-hackathon/test/labels' copied successfully to '/kaggle/working/crop_34_bands/test/labels'.


In [13]:
base_dir = '/kaggle/input/crop-dataset/track1-india-al-impact-gen-ai-hackathon'
out_dir = '/kaggle/working/crop_34_bands' # Changed output dir name for clarity
sub_dirs = ['train.txt', 'val.txt', 'test.txt']

for sub in sub_dirs:
    input_directory = os.path.join(base_dir, sub)
    output_directory = os.path.join(out_dir, sub)

    print("-" * 50)
    
    shutil.copyfile(input_directory, output_directory)
    print("\nAll script operations completed!")


--------------------------------------------------

All script operations completed!
--------------------------------------------------

All script operations completed!
--------------------------------------------------

All script operations completed!


In [14]:
dataset_path = '/kaggle/working/crop_34_bands/'

datamodule = GenericNonGeoSegmentationDataModule(
    batch_size=10,
    num_workers=2,
    num_classes=6,

    # Define dataset paths
    train_data_root=dataset_path+'/train/inputs',
    train_label_data_root=dataset_path+'/train/labels',
    val_data_root=dataset_path+'/val/inputs',
    val_label_data_root=dataset_path+'/val/labels',
    test_data_root=dataset_path+'/val/inputs',
    test_label_data_root=dataset_path+'/val/labels',
    
    #data set path for infereencing on test input
    predict_data_root=dataset_path+'test/inputs',
 
    # Define splits
    train_split=dataset_path+"/train.txt",
    val_split=dataset_path+"/val.txt",
    test_split=dataset_path+"/val.txt",

    
    img_grep='*input.tif',
    label_grep='*label_c6.tif',

    train_transform=[
        albumentations.D4(),
        ToTensorV2(),
    ],
    val_transform=None,
    test_transform=None,
    means = [43.414087986123974, 38.81009859874331, 37.657296657562256, 39.46452532143429, 42.66194085417123, 54.86814571249074, 63.38708205058657, 60.08845993568157, 13.279416692667994, 69.287048570041, 48.36614383500198, 69.84629751073903, 0.2132587335839987, 26370903.853171803, 0.3182134306061422, 0.13085361243484275, -0.07412845801290406, 0.17232999177350547, 0.04472540249474705, -0.017443628418975062, 0.004647128527066735, 0.22877457022055528, 5956769.690392101, 0.21290016207165716, -0.7262815527178053, 2.638937688962349, 6.315807526144584, -312.4690136898994, 61684051.37808944, 0.4463677766159984, -372476.07332120626, -0.07555472765309695, -2.9600306050530794, 2.0925896116181475],
    stds = [3.249200333906643, 4.092319172521173, 5.404914137661227, 9.249756503354499, 8.044317368830914, 6.790147526536125, 8.10124124355119, 7.866002415825067, 2.4995222018345795, 17.069685938871878, 15.658058630599031, 9.297859936811728, 0.1108298246651591, 410681869.75464195, 0.16528027862984687, 0.1735479324223879, 0.018316452678591062, 0.08676941487768036, 0.027369521891671803, 0.030296780356157682, 0.12347729336499248, 0.07469651735070214, 223392898.5723597, 0.1106232296227927, 0.05979664023763734, 3.251688884134678, 7.749837612234008, 66721.16934437146, 229531054.1366095, 0.28962555049655164, 20673357.31535897, 0.1297231175522108, 3.3168257425479246, 1.033343191206589],
    no_data_replace=0,
    no_label_replace=-1,
)


datamodule.setup("fit")
datamodule.setup("predict")

In [15]:

print("Train set size:", len(datamodule.train_dataset))
print("Validation set size:", len(datamodule.val_dataset))

datamodule.setup("test")
print("Test set size:", len(datamodule.test_dataset))

Train set size: 928
Validation set size: 119
Test set size: 119


In [16]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

OUT_DIR = "/kaggle/working"

BATCH_SIZE = 16

EPOCHS = 120
LR = 1e-5
WEIGHT_DECAY = 0.01
HEAD_DROPOUT = 0.25
FREEZE_BACKBONE = False

BANDS =[
        "B01",
        "BLUE",
        "GREEN",
        "RED",
        "B05",
        "B06",
        "B07",
        "NIR_NARROW",
        "B8A",
        "B09",
        "SWIR_1",
        "SWIR_2",
        'NDVI',   
        'EVI',    
        'SAVI',   
        'NDMI',   
        'GVMI',   
        'NDRE1', 
        'NDRE2', 
        'NDYI',  
        'PSRI',  
        'GNDVI', 
        'REP',
        'OSAVI',
        'WDRVI',
        'MCARI',
        'TCARI',
        'CCCI',
        'MTCI',
        'CIre', 
        'SIPI', 
        'BSI',
        'EXG',
        'RDVI', 
    ]
NUM_FRAMES = 1

#      Crop     Pixel_total    Pixel_percentage Class_weight
#      Gram        2545             1.73       0.4761
#     Maize        7128             4.84       0.1702
#   Mustard       36362            24.67       0.0334
# Sugarcane        4542             3.08       0.2674
#     Wheat       59585            40.42       0.0204
# OtherCrop       37247            25.27       0.0326

CLASS_WEIGHTS = [0.40, 0.15, 0.05, 0.35, 0.03, 0.04]

SEED = 0
pl.seed_everything(SEED)

INFO: Seed set to 0


0

In [17]:
# from lightning.pytorch.callbacks import LearningRateMonitor
# lr_monitor = LearningRateMonitor(logging_interval='epoch')

In [18]:
# SEED = 0

# pl.seed_everything(SEED)

# # Logger
# logger = TensorBoardLogger(
#     save_dir=OUT_DIR,
#     name="crop",
# )

# # Callbacks
# checkpoint_callback = ModelCheckpoint(
#     monitor="val/Multiclass_Jaccard_Index",
#     mode="max",
#     dirpath=os.path.join(OUT_DIR, "cropid", "checkpoints"),
#     filename="best-checkpoint-{epoch:02d}-{val_loss:.2f}",
#     save_top_k=1,
#     verbose=True
# )

In [19]:
SEED = 0

pl.seed_everything(SEED)

# Logger
logger = TensorBoardLogger(
    save_dir=OUT_DIR,
    name="crop",
)

# Callbacks
checkpoint_callback = ModelCheckpoint(
    monitor="val/Multiclass_Jaccard_Index",
    mode="max",
    dirpath=os.path.join(OUT_DIR, "cropid", "checkpoints"),
    filename="best-checkpoint-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,
    verbose=True
)

INFO: Seed set to 0


In [20]:
# rm -rf /kaggle/working/crop_17_bands

In [23]:
# Correct imports
from terratorch.registry import TERRATORCH_NECK_REGISTRY
import torch
import torch.nn as nn
import torch.nn.functional as F



In [24]:
class FocalTverskyLoss(nn.Module):
    """
    Focal Tversky Loss for imbalanced multi-class semantic segmentation.
    Applies focal mechanism directly to Tversky Index, with class weighting.
    """
    def __init__(self, num_classes, class_weights, alpha=0.3, beta=0.7, gamma=4/3, epsilon=1e-7, ignore_index=-1):
        super(FocalTverskyLoss, self).__init__()
        self.num_classes = num_classes
        self.alpha = alpha  # Weight on FP (lower to improve recall for rare classes)
        self.beta = beta    # Weight on FN (higher to penalize missing rare class pixels)
        self.gamma = gamma  # Focal parameter (>1 to focus on hard examples)
        self.epsilon = epsilon
        self.ignore_index = ignore_index
        
        if not isinstance(class_weights, torch.Tensor):
            class_weights = torch.tensor(class_weights, dtype=torch.float32)
        self.register_buffer('class_weights', class_weights)  # Register as buffer to move with module.to(device)

    def forward(self, inputs, targets):
        # inputs shape: [B, C, H, W], targets shape: [B, H, W]
        
        # Reshape and filter ignored pixels
        logits_flat = inputs.permute(0, 2, 3, 1).contiguous().view(-1, self.num_classes)
        targets_flat = targets.view(-1)
        
        valid_mask = targets_flat != self.ignore_index
        logits_flat = logits_flat[valid_mask]
        targets_flat = targets_flat[valid_mask]

        if targets_flat.numel() == 0:
            return inputs.sum() * 0.0

        probs = F.softmax(logits_flat, dim=1)
        targets_one_hot = F.one_hot(targets_flat, num_classes=self.num_classes).float()
        
        # Compute per-class sums (across all pixels)
        tp = torch.sum(probs * targets_one_hot, dim=0)
        fp = torch.sum(probs * (1 - targets_one_hot), dim=0)
        fn = torch.sum((1 - probs) * targets_one_hot, dim=0)
        
        # Per-class Tversky Index
        tversky_index = (tp + self.epsilon) / (tp + self.alpha * fp + self.beta * fn + self.epsilon)
        
        # Focal Tversky per class: (1 - TI)^{1/gamma}
        ftl_per_class = (1 - tversky_index) ** (1 / self.gamma)
        
        # Weighted average across classes
        weighted_ftl = torch.sum(self.class_weights * ftl_per_class) / torch.sum(self.class_weights)
        
        return weighted_ftl

In [33]:
from terratorch.tasks import SemanticSegmentationTask

class CustomSegmentationTaskWithEQLv2(SemanticSegmentationTask):
    def configure_losses(self) -> None:
        """
        This method overrides the original loss configuration from the parent class.
        It now also passes the correct 'ignore_index' to the EQLv2 loss.
        """
        # cls_num_list = [2545, 7128, 36362, 4542, 59585, 37247]
        # num_classes = self.hparams.model_args["num_classes"]
        # # ignore_index = self.hparams.get("ignore_index", -1)
        # alpha = torch.tensor([
        #     50.0,   # Class 0 - rarest crop (2545 pixels) 
        #     25.0,   # Class 1 - (7128 pixels)
        #     10.0,    # Class 2 - (36362 pixels)
        #     40.0,   # Class 3 - (4542 pixels) 
        #     8.0,    # Class 4 - most common crop (59585 pixels)
        #     10.0    # Class 5 - (37247 pixels)
        # ], dtype=torch.float32)
        
        # self.criterion = FocalDiceLoss(
        #                 alpha=alpha,
        #                 gamma=2.5,          # Higher gamma for extreme imbalance
        #                 focal_weight=0.2,   # Emphasize focal loss for imbalance
        #                 dice_weight=0.8,    # Lower dice weight
        #             )
        CLASS_WEIGHTS = [0.40, 0.15, 0.05, 0.35, 0.03, 0.04]
        NUM_CLASSES = 6
        
        self.criterion = FocalTverskyLoss(
            num_classes=NUM_CLASSES, 
            class_weights=CLASS_WEIGHTS,
            alpha=0.3,
            beta=0.7,
            gamma=4/3
        )


    def on_train_epoch_end(self):
        # Get current LR from the optimizer
        current_lr = self.trainer.optimizers[0].param_groups[0]['lr']
        # Log it (will show in console and TensorBoard)
        self.log('lr', current_lr, prog_bar=True, logger=True)
        print(f"INFO: Epoch {self.current_epoch}, Current LR: {current_lr:.6f}")
        
        # Call super to keep original behavior
        super().on_train_epoch_end()

In [34]:
# Correct imports
from terratorch.registry import TERRATORCH_NECK_REGISTRY
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the neck class
class CropVIAttentionNeck(nn.Module):
    """Crop-specific VI attention neck for Prithvi model"""
    
    def __init__(self, in_channels, num_classes=6):
        super().__init__()
        
        # Handle in_channels as list
        if isinstance(in_channels, int):
            in_channels = [in_channels]
        self.in_channels_list = in_channels
        
        # Your exact VI importance weights
        self.crop_vi_importance = {
            0: [0.9, 0.8, 0.8, 0.8, 0.8, 0.7, 0.7, 0.0, 0.6, 0.8, 0.7, 0.9, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.3, 0.5, 0.8],  # Gram
            1: [0.9, 1.0, 0.7, 0.7, 0.7, 0.8, 0.8, 0.0, 0.6, 0.8, 0.7, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.9, 0.7, 0.3, 0.5, 0.9],  # Maize  
            2: [0.8, 0.8, 0.6, 0.7, 0.7, 0.6, 0.6, 1.0, 0.7, 0.9, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.7, 0.3, 0.6, 0.7],  # Mustard
            3: [1.0, 0.9, 0.7, 0.8, 0.8, 0.7, 0.7, 0.0, 0.6, 0.8, 0.7, 0.8, 0.8, 0.7, 0.7, 0.7, 0.7, 0.7, 0.6, 0.3, 0.5, 0.8],  # Sugarcane
            4: [0.9, 0.8, 0.7, 0.7, 0.7, 0.8, 0.7, 0.0, 0.6, 1.0, 0.8, 0.8, 0.9, 0.7, 0.7, 0.7, 0.8, 0.8, 0.7, 0.3, 0.5, 0.8],  # Wheat
            5: [0.8, 0.7, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.7, 0.6, 0.5, 0.5, 0.4, 0.4, 0.4, 0.5, 0.4, 0.4, 0.3, 0.4, 0.5]   # Other
        }
        
        # Convert to tensor [6, 11] for VI bands
        vi_weights = torch.tensor([self.crop_vi_importance[i] for i in range(6)], dtype=torch.float32)
        self.register_buffer('vi_importance_weights', vi_weights)
        
        # Attention layers per level, avoiding inplace
        self.attention_layers = nn.ModuleList()
        for ch in self.in_channels_list:
            reduced_ch = max(ch // 4, 1)
            self.attention_layers.append(nn.Sequential(
                nn.Linear(ch, reduced_ch),
                nn.ReLU(inplace=False),  # Explicitly out-of-place
                nn.Linear(reduced_ch, ch),
                nn.Sigmoid()
            ))
        
    def process_channel_list(self, in_channels: list[int]) -> list[int]:
        """Required by TerraTorch - no channel changes"""
        return in_channels
        
    def forward(self, inputs: list[torch.Tensor]) -> list[torch.Tensor]:
        """Forward pass - handles list of token features [B, N, D]"""
        outputs = []
        for level_idx, x in enumerate(inputs):
            # x is [B, N, D] for tokens
            B, N, D = x.shape
            
            # Global average pooling over tokens (out-of-place)
            global_features = torch.mean(x, dim=1)  # [B, D]
            
            # Channel attention (on D dimension, out-of-place)
            channel_weights = self.attention_layers[level_idx](global_features)  # [B, D]
            
            # Apply VI importance (out-of-place with clone)
            avg_vi_importance = torch.mean(self.vi_importance_weights, dim=0)  # [11]
            if D >= 11:
                vi_weights = channel_weights[:, -22:].clone() * avg_vi_importance
                channel_weights = channel_weights.clone()
                channel_weights[:, -22:] = vi_weights
            
            # Reshape and apply attention (out-of-place)
            attention = channel_weights.unsqueeze(1)  # [B, 1, D]
            attended_features = x * attention
            
            outputs.append(attended_features)
        
        return outputs

# Define a constructor function
def create_crop_vi_attention_neck(in_channels=[1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024]
, num_classes=6, **kwargs):
    """Constructor for CropVIAttentionNeck"""
    return CropVIAttentionNeck(in_channels=in_channels, num_classes=num_classes, **kwargs)

# Register it
TERRATORCH_NECK_REGISTRY.register( create_crop_vi_attention_neck)

print("✅ Custom neck registered with TerraTorch!")


✅ Custom neck registered with TerraTorch!


In [35]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# # Correct import
# from terratorch.registry import NECK_REGISTRY

# # Register the custom neck
# @NECK_REGISTRY.register()
# class CropVIAttentionNeck(nn.Module):
#     """Crop-specific VI attention neck for Prithvi model"""
    
#     def __init__(self, in_channels=17, num_classes=6):
#         super().__init__()
        
#         # Your exact VI importance weights
#         self.crop_vi_importance = {
#             0: [0.9, 0.8, 0.8, 0.8, 0.8, 0.7, 0.7, 0.0, 0.6, 0.8, 0.7],  # Gram
#             1: [0.9, 1.0, 0.7, 0.7, 0.7, 0.8, 0.8, 0.0, 0.6, 0.8, 0.7],  # Maize  
#             2: [0.8, 0.8, 0.6, 0.7, 0.7, 0.6, 0.6, 1.0, 0.7, 0.9, 0.7],  # Mustard
#             3: [1.0, 0.9, 0.7, 0.8, 0.8, 0.7, 0.7, 0.0, 0.6, 0.8, 0.7],  # Sugarcane
#             4: [0.9, 0.8, 0.7, 0.7, 0.7, 0.8, 0.7, 0.0, 0.6, 1.0, 0.8],  # Wheat
#             5: [0.8, 0.7, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.7, 0.6]   # Other
#         }
        
#         # Convert to tensor [6, 11] for VI bands (indices 6-16)
#         vi_weights = torch.tensor([self.crop_vi_importance[i] for i in range(6)], dtype=torch.float32)
#         self.register_buffer('vi_importance_weights', vi_weights)
        
#         # Simple attention mechanism
#         self.global_pool = nn.AdaptiveAvgPool2d(1)
#         self.channel_attention = nn.Sequential(
#             nn.Linear(in_channels, max(in_channels // 4, 1)),
#             nn.ReLU(),
#             nn.Linear(max(in_channels // 4, 1), in_channels),
#             nn.Sigmoid()
#         )
        
#     def forward(self, x):
#         B, C, H, W = x.shape
        
#         # Global average pooling for channel attention
#         global_features = self.global_pool(x).squeeze(-1).squeeze(-1)  # [B, 17]
        
#         # Base channel attention
#         channel_weights = self.channel_attention(global_features)  # [B, 17]
        
#         # Apply VI importance to vegetation index bands (6-16)
#         avg_vi_importance = torch.mean(self.vi_importance_weights, dim=0)  # [11]
#         channel_weights[:, 6:] = channel_weights[:, 6:] * avg_vi_importance
        
#         # Reshape and apply attention
#         attention = channel_weights.unsqueeze(-1).unsqueeze(-1)  # [B, 17, 1, 1]
#         attended_features = x * attention
        
#         return attended_features


# =============================================================================
# MODIFY YOUR MODEL ARGS - Replace your neck definition
# =============================================================================
data_module = datamodule
# Keep everything exactly the same, just modify the necks part
backbone_args = dict(
    backbone_pretrained=True,
    backbone="prithvi_eo_v2_600_tl",  # Your exact backbone
    backbone_bands=BANDS,  # Your exact bands
    backbone_num_frames=1,
)

decoder_args = dict(
    decoder="UperNetDecoder",  # Your exact decoder
    decoder_channels=256,
    decoder_scale_modules=True
)

# MODIFIED: Add crop attention neck before the reshape neck
necks = [
    # First apply crop attentionxq
    dict(
        name="create_crop_vi_attention_neck",  # Use the registered class name
        
    ),
    # Then reshape as before
    dict(
        name="ReshapeTokensToImage",
        effective_time_dim=NUM_FRAMES,
    )
]
# Keep your exact model args, just with new necks
model_args = dict(
    **backbone_args,
    **decoder_args,
    num_classes=6,  # Your exact classes
    head_dropout=HEAD_DROPOUT,  # Your exact dropout
    necks=necks,  # Modified necks with attention
    rescale=True
)

# =============================================================================
# KEEP YOUR EXACT MODEL CREATION - No changes needed!
# =============================================================================

model = CustomSegmentationTaskWithEQLv2(
    model_args=model_args,
    plot_on_val=False,
    class_weights=CLASS_WEIGHTS,  # Your exact class weights
    # loss="dice",  # Your exact loss
    lr=LR,  # Your exact learning rate
    optimizer="AdamW",  # Your exact optimizer
    optimizer_hparams=dict(weight_decay=WEIGHT_DECAY),  # Your exact weight decay
    freeze_backbone=FREEZE_BACKBONE,  # Your exact freeze setting
    freeze_decoder=False,
    model_factory="EncoderDecoderFactory",  # Your exact factory
    ignore_index=-1,
    scheduler='CosineAnnealingLR',
    scheduler_hparams={
        "T_max": 120,
        "eta_min": 1e-8
    }
)

print("✅ Prithvi model enhanced with crop-specific VI attention!")
print("✅ All your original settings preserved")
print("✅ Attention applied to VI bands based on crop importance")

# =============================================================================
# TRAINING - Keep exactly the same!
# =============================================================================


✅ Prithvi model enhanced with crop-specific VI attention!
✅ All your original settings preserved
✅ Attention applied to VI bands based on crop importance


In [ ]:
# import torch
# import gc
# torch.cuda.empty_cache()
# gc.collect()


In [ ]:

trainer = pl.Trainer(
    accelerator="auto",
    strategy="auto",
    devices="auto",
    precision="16-mixed",
    num_nodes=1,
    logger=logger,
    max_epochs=EPOCHS,
    check_val_every_n_epoch=1,
    log_every_n_steps=5,
    accumulate_grad_batches=3,
    enable_checkpointing=True,
    callbacks=[checkpoint_callback],
)

trainer.fit(model, datamodule=datamodule)


INFO: Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: 
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | PixelWiseModel   | 683 M  | train
1 | criterion     | FocalTverskyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | val_metrics   | MetricCollection | 0      | train
4 | test_metrics  | ModuleList       | 0      | train
-----------------------------------------------------------
683 M     Trainable params
0 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 31: 'val/Multiclass_Jaccard_Index' reached 0.18225 (best 0.18225), saving model to '/kaggle/working/cropid/checkpoints/best-checkpoint-epoch=00-val_loss=0.00.ckpt' as top 1


...

INFO: Epoch 62, global step 1953: 'val/Multiclass_Jaccard_Index' was not in top 1


INFO: Epoch 62, Current LR: 0.000005


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 63, global step 1984: 'val/Multiclass_Jaccard_Index' was not in top 1


INFO: Epoch 63, Current LR: 0.000004


In [ ]:
trainer.test(dataloaders=datamodule)

In [ ]:
import os
import zipfile

def zip_folder(folder_path, output_path):
    """
    Zips the entire contents of a folder.

    Args:
        folder_path (str): The path to the folder you want to zip.
        output_path (str): The path for the output zip file.
    """
    try:
        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # os.walk() generates the file names in a directory tree
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    # Create the full path of the file
                    file_path = os.path.join(root, file)
                    # Create a relative path for the file inside the zip archive
                    # This keeps the folder structure intact without the top-level path
                    archive_path = os.path.relpath(file_path, folder_path)
                    zipf.write(file_path, arcname=archive_path)
        
        print(f"Successfully created zip file: {output_path} ✅")

    except Exception as e:
        print(f"An error occurred: {e}")


# --- Main execution ---
if __name__ == "__main__":
    # 1. Define the folder you want to zip
    folder_to_zip = '/kaggle/working/crop'
    
    # 2. Define the name and path for the output zip file
    output_zip_filename = '/kaggle/working/crop_log.zip'
        
    # 3. Call the function to zip the folder
    zip_folder(folder_to_zip, output_zip_filename)

In [ ]:

checkpoint_dir = "/kaggle/working/cropid/checkpoints"
checkpoint_files = glob.glob(os.path.join(checkpoint_dir, "best-checkpoint-*.ckpt"))

if len(checkpoint_files) == 0:
    raise FileNotFoundError("No best checkpoint file found in the directory.")
    
# Use the first match
best_ckpt_path = checkpoint_files[0]
# test_results = trainer.test(model, datamodule=datamodule, ckpt_path=best_ckpt_path)
# print(test_results)
print(best_ckpt_path)


In [ ]:
# A. Saving predictions on test input images  
 
# Prediction on predict_data_root (see dataloader part)
preds = trainer.predict(model, datamodule=datamodule, ckpt_path=best_ckpt_path)

output_dataset_path = "/kaggle/working/"
# Output directory to save prediction tif files
output_dir = os.path.join(output_dataset_path, "test_pred")
os.makedirs(output_dir, exist_ok=True)
 
# Loop over batches
for batch_idx, batch in enumerate(preds):
    tensor = batch[0][0]    # shape [4, 256, 256] → 4 images in this batch
    file_paths = batch[1]   # list of file paths for this batch
 
    for i in range(tensor.shape[0]):  # loop over each file in batch
        arr = tensor[i].cpu().numpy().astype('int32')  # shape [256, 256] → single band
 
        ref_path = file_paths[i]
        with rasterio.open(ref_path) as src_ref:
            ref_crs = src_ref.crs
            ref_transform = src_ref.transform
 
        out_name = os.path.splitext(os.path.basename(ref_path))[0] + "_pred.tif"
        out_path = os.path.join(output_dir, out_name)
 
        with rasterio.open(
            out_path,
            "w",
            driver="GTiff",
            height=arr.shape[0],
            width=arr.shape[1],
            count=1,
            dtype=arr.dtype,
            crs=ref_crs,
            transform=ref_transform
        ) as dst:
            dst.write(arr, 1)
 
        print(f"Saved {out_path}")

In [ ]:
# Input directories
# dir_pred = os.path.join(dataset_path, "test_pred")
# dir_mask = os.path.join(dataset_path, "test")
# prediction_csv = os.path.join(dataset_path, "submission.csv")
 
# dir_pred = os.path.join(dataset_path, "test")
# dir_mask = os.path.join(OUT_DIR, "")
# prediction_csv = os.path.join(OUT_DIR, "test_labels_2.csv")
 
# Input directories
dir_pred = os.path.join(output_dataset_path, "test_pred")
dir_mask = os.path.join(dataset_path, "test/labels")
prediction_csv = os.path.join(output_dataset_path, "prediction_latest1.csv")
 
def mask_to_rle(mask):   
    mask = mask.flatten(order='F')
    pixels = np.concatenate([[0], mask, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)
 
 
records = []
 
# Get prediction files
pred_files = glob.glob(os.path.join(dir_pred, "*.tif"))
# pred_files = glob.glob(os.path.join(dir_pred, "*label_c6.tif"))
 
for pred_file in pred_files:
    base_name = os.path.basename(pred_file)
    mask_file = os.path.join(dir_mask, base_name.replace("input_pred.tif", "label_mask.tif"))
    # mask_file = os.path.join(dir_mask, base_name.replace("label_c6.tif", "label_mask.tif"))
    
    if not os.path.exists(mask_file):
        print(f"No mask found for {base_name}, skipping")
        continue
 
    # Read prediction
    with rasterio.open(pred_file) as src_pred:
        pred_data = src_pred.read(1)
 
    # Read mask
    with rasterio.open(mask_file) as src_mask:
        mask_data = src_mask.read(1)
 
    # Apply mask (only keep pixels where mask == 1)
    valid_idx = np.where(mask_data == 1)
 
    # Pixel IDs
    pixel_ids = np.ravel_multi_index(valid_idx, pred_data.shape)
 
    # Predictions on masked pixels
    masked_preds = pred_data[valid_idx]
    # masked_preds = pred_data
 
    # For each class (0–5)
    flat_size = pred_data.size
    for cls in range(6):
        cls_mask = np.zeros(flat_size, dtype=np.uint8)
        cls_mask[pixel_ids[masked_preds == cls]] = 1
        # Select pixels belonging to this class
        # cls_pixels = pixel_ids[masked_preds == cls]
 
        # if cls_pixels.size > 0:
        #     # Format as "pixel_id class" pairs
        #     pred_str = " ".join(f"{pid} {cls}" for pid in cls_pixels.tolist())
        # else:
        #     pred_str = ""
        if cls_mask.sum() > 0:
            pred_str = mask_to_rle(cls_mask)
        else:
            pred_str = "0"
 
        # Remove "_input_pred.tif" and add class as suffix
        file_id = base_name.replace("_input_pred.tif", f"_{cls}")
        # file_id = base_name.replace("_label_c6.tif", f"_{cls}")
 
        records.append([file_id, pred_str])
 
# Create df
df = pd.DataFrame(records, columns=["id", "label"])

#df["label"] = df["label"].fillna("0")
 
# Save CSV
df.to_csv(prediction_csv, index=False)
 
print(f"Saved predictions to {prediction_csv}")
 